In [1]:
import torch
import os
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
from torchvision.utils import save_image
from PIL import Image
from scipy.io import loadmat

class FreyFaceDataset(torch.utils.data.Dataset):
    # data_file: available at https://cs.nyu.edu/~roweis/data/frey_rawface.mat
    data_file = 'frey_rawface.mat'

    def __init__(self, root, transform=None):
        super(FreyFaceDataset, self).__init__()
        self.root = root
        self.transform = transform
        if not self._check_exists():
            raise RuntimeError('Dataset do not found in the directory \"{}\".'.format(self.root))
        self.data = loadmat(os.path.join(self.root, self.data_file))['ff'].T

    def __getitem__(self, index):
        img = self.data[index].reshape(28, 20)
        img = Image.fromarray(img, mode='L')
        if self.transform is not None:
            img = self.transform(img)
        return img

    def __len__(self):
        return len(self.data)

    def _check_exists(self):
        return os.path.exists(os.path.join(self.root, self.data_file))

entire_dataset = FreyFaceDataset(root='/root/data/frey', transform=transforms.ToTensor())

bs = 100
indices = torch.randperm(len(entire_dataset)).tolist()
test_indices, train_indices = indices[:400], indices[400:]
train_dataset = torch.utils.data.Subset(dataset=entire_dataset, indices=train_indices)
test_dataset = torch.utils.data.Subset(dataset=entire_dataset, indices=test_indices)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=bs, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=bs, shuffle=False)



In [2]:
import torch
import torch.nn as nn


class VAE(nn.Module):
    def __init__(self, input_size, hidden_size, latent_size):
        super(VAE, self).__init__()
        # Encoder: layers
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc21 = nn.Linear(hidden_size, latent_size)
        self.fc22 = nn.Linear(hidden_size, latent_size)
        # Decoder: layers
        self.fc3 = nn.Linear(latent_size, hidden_size)
        self.fc41 = nn.Linear(hidden_size, input_size)
        self.fc42 = nn.Linear(hidden_size, input_size)

    def encode(self, x):
        h1 = torch.tanh(self.fc1(x))
        mean, log_var = self.fc21(h1), self.fc22(h1)
        return mean, log_var

    def reparameterize(self, mean, log_var):
        mu, std = mean, torch.exp(0.5 * log_var)
        epsilon = torch.randn_like(std)
        z = mu + std * epsilon
        return z

    def decode(self, z):
        h3 = torch.tanh(self.fc3(z))
        mean, log_var = torch.sigmoid(self.fc41(h3)), self.fc42(h3)
        return mean, log_var

    def forward(self, x):
        z_mean, z_logvar = self.encode(x)
        z = self.reparameterize(z_mean, z_logvar)
        return z_mean, z_logvar, self.decode(z)

In [91]:
learning_rate = 1e-3
input_size = 560
hidden_size = 200
latent_size = 2

from time import time

n_epochs = 1000

pi = torch.tensor(torch.pi)

def compute_elbo(x, mean, log_var, out_mean, out_logvar):
    
    reconst_error = -0.5 * torch.sum(torch.log(2*pi) + out_logvar + (x - out_mean).pow(2) / out_logvar.exp())
    
    kl_divergence = -0.5 * torch.sum(1 + log_var - mean.pow(2) - log_var.exp())
    elbo = (reconst_error - kl_divergence) / len(x)
    return elbo

def train(device, model, latent_dim):
    print(latent_dim)
    #optimizer = torch.optim.Adagrad(model.parameters(), lr=learning_rate)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    begin_time = time()
    # train
    loss_list = []
    with open('/root/paper/elbo/frey_2.txt', 'w') as file:
        for i in range(n_epochs):
            loss_sum = 0
            for batch_idx, x in enumerate(train_loader):
                x = x.to(device).view(-1, input_size)
                #x = x.to(device)
                mean, logvar, (output_mean, output_logvar) = model(x)
                loss = -compute_elbo(x, mean, logvar, output_mean, output_logvar)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                #loss_sum += loss.item()*len(x)
                '''
                if batch_idx % 100 == 0:
                    print(torch.min(output_logvar).item())
                    print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(i, batch_idx * len(x), len(train_loader.dataset),100. * batch_idx / len(train_loader), loss.item()))
                '''
            # estimate elbo
            model.eval()
            with torch.no_grad():
                n_samples = 100
                #train
                indices = torch.randperm(len(train_dataset))[:n_samples]
                x = torch.stack([train_dataset[i][0] for i in indices]).to(device).view(-1, input_size)
                mean, logvar, (output_mean, output_logvar) = model(x)
                loss_sum = -compute_elbo(x, mean, logvar, output_mean, output_logvar)
                if(i % 10 == 0):
                    print('====> Epoch: {} elbo loss: {:.4f}'.format(i, loss_sum))
                file.write(str(-loss_sum.item()) + ' ')
                #test
                indices = torch.randperm(len(test_dataset))[:n_samples]
                x = torch.stack([test_dataset[i][0] for i in indices]).to(device).view(-1, input_size)
                mean, logvar, (output_mean, output_logvar) = model(x)
                loss_sum = -compute_elbo(x, mean, logvar, output_mean, output_logvar)
                if(i % 10 == 0):
                    print('====> Epoch: {} elbo loss: {:.4f}'.format(i, loss_sum))
                file.write(str(-loss_sum.item()) + '\n')
                
            
            #time
            if(i % 10 == 0):
                training_time = time() - begin_time
                minute = int(training_time // 60)
                second = int(training_time % 60)
                print(f'time loss {minute}:{second}')
                #torch.save(model, '/root/VAE/VAE_model.pth')
    tot_training_time = time() - begin_time
    minute = int(tot_training_time // 60)
    second = int(tot_training_time % 60)
    print(f'total time loss {minute}:{second}')
    
    
def initialize_parameters(model):
    for param in model.parameters():
        param.data.normal_(mean=0, std=0.01)

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from scipy.interpolate import interp1d
        
def moving_average(data, window_size):
    """Compute moving average."""
    moving_averages = []
    for i in range(len(data) - window_size + 1):
        if(i > 20):
            window = data[i:i+window_size]
            moving_averages.append(sum(window) / window_size)
        else:
            moving_averages.append(data[i])
    return moving_averages

def drawing():
    plt.clf()
    fig = plt.figure(figsize=(3, 3))
    trains = []
    tests = []
    with open('/root/paper/elbo/frey_20.txt', 'r') as file:
        for line in file:
            parts = line.split()
            trains.append(float(parts[0]))
            tests.append(float(parts[1]))
            
    epochs = [len(train_dataset) * i for i in range(1, len(trains) + 1)]

    window_size = 10 
    trains_smoothed = moving_average(trains, window_size)
    tests_smoothed = moving_average(tests, window_size)
    epochs_smoothed = moving_average(epochs, window_size)

    plt.plot(epochs_smoothed, trains_smoothed, label='Train')
    plt.plot(epochs_smoothed, tests_smoothed, label='Test')
    plt.xlabel('Samples')
    plt.ylabel('ELBO')
    plt.title('Frey Face, d=20')
    plt.legend()
    plt.ylim(0, 1600)
    plt.xlim(1e3, len(train_dataset)*len(trains))
    plt.xscale('log')
    plt.savefig('/root/paper/elbo/pictures/frey_20.png',bbox_inches='tight')
    plt.show()
    

In [ ]:
def main():
    device = 'cuda:0'

    #model = VAE(latent_dim = 10).to(device)

    # Load the model
    #model = torch.load('/root/VAE/VAE_model.pth', map_location=device)

    #Choose which to play
    model = VAE(input_size, hidden_size, latent_size).to(device)
    initialize_parameters(model)
    train(device, model, latent_size)
    #generate(device, model)
    #reconstruct(device, model)
    #latent_space(device, model)
    #explore_latent()
    #print(sum(p.numel() for p in model.parameters() if p.requires_grad))

if __name__ == '__main__':
    main()

In [ ]:
drawing()